In [ ]:
import pandas as pd

# df = pd.read_csv('data/benin-malanville.csv')  # Make sure this 
# file exists
df = pd.read_csv("data/togo-dapaong_qc.csv", parse_dates=["Timestamp"])
df.head()


In [ ]:
df.describe()
df.isna().sum().sort_values(ascending=False)

In [ ]:
from scipy.stats import zscore

for col in ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']:
    if col in df.columns:
        df['z_' + col] = zscore(df[col])
        print(f"{col} outliers:", (df['z_' + col].abs() > 3).sum())


In [ ]:
# Drop rows where key columns are missing
df = df.dropna(subset=['GHI', 'DNI', 'DHI'])

# Fill missing sensor values with median
for col in ['ModA', 'ModB', 'WS', 'WSgust']:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Drop z-score columns (if any)
df.drop(columns=[col for col in df.columns if str(col).startswith('z_')], inplace=True)


In [ ]:
df.to_csv('data/togo-dapaong_qc.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt

df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # Adjust if your time column is named differently
df.set_index('Timestamp').resample('D')[['GHI', 'DNI', 'DHI']].mean().plot()
plt.title('Daily Solar Radiation')
plt.show()
